In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import rcca_modified as rcca
#reload(rcca)
from unils_cca import set_diag_to_zero
from metrics import ClassificationResult
from unils_cca import binarize, set_diag_to_zero, BuildMatrixA, printMatrix, print_q_val_hist, zscore, plot_zscore, BuildFeatureVector,\
train_test_2D, build_distance_for_node,  get_features, build_distance_matrix, remove_ambiguous, \
get_pairs_distance_matched,  reshape_2d_to_1d, demean, get_pos_neg_y_and_ypred_at_thres, subsample_neg
import copy
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
InteractionsFileCaptureC='/srv/scratch/mtaranov/CaptureC_ChicagoCalls/CaptureC_SC_bait-bait.bed.gz'
#InteractionsFileCaptureC='/mnt/lab_data/kundaje/mtaranov/ChicagoCalls/D0_D2D8_merge_BaitToBait_intra.bed.gz'
PromoterFile='/srv/scratch/mtaranov/HindIII/PromoterCapture_Digest_Human_HindIII_baits_ID.bed'
DATADIR='/users/mtaranov/CCA_model/data/'
DATADIR_CaptureC = DATADIR+'CaptureC/'
DATADIR_kernels = DATADIR+'CaptureC_kernels/'
DATADIR_nodes = DATADIR+'nodes/'
DATADIR_features = DATADIR+'features/'
DATADIR_preds = DATADIR+'preds_2d/'
DATADIR_weights = DATADIR+'weights/'

## Train model with kernel_CaptureC, save weights

In [21]:
all_chrs = ['chr'+str(i+1) for i in range(22)]+['chrX']

In [22]:
best_reg=0.001
best_numCC=50
cca = rcca.CCA(reg=best_reg, numCC=best_numCC)

for chr in all_chrs:
    kernel_CaptureC = np.load(DATADIR_kernels+'kernel_CaptureC_'+chr+'.npy')
    features = np.load(DATADIR_features+'features_'+chr+'.npy')
    cca_train = cca.train([demean(kernel_CaptureC), demean(features)])
    np.save(DATADIR_weights+'ws_'+chr, cca_train.__dict__['ws'][1])

Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 comp

## Save predictions, 2d

In [23]:
for chr in all_chrs:
    print chr
    without_current_chr = copy.copy(all_chrs)
    without_current_chr.remove(chr)
    ws_all = []
    for other_chr in without_current_chr:
        ws_all.append(np.load(DATADIR_weights+'ws_'+other_chr+'.npy'))
    ws = sum(ws_all)/len(without_current_chr) 
    features = np.load(DATADIR_features+'features_'+chr+'.npy')
    preds, projs = cca.predict_from_features(features, ws)
    np.save(DATADIR_preds+'pred_2d_'+chr, set_diag_to_zero(preds))

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX


## Train model with raw CaptureC, save weights

In [24]:
best_reg=0.001
best_numCC=50
cca = rcca.CCA(reg=best_reg, numCC=best_numCC)

for chr in all_chrs:
    CaptureC = np.load(DATADIR_CaptureC+'CaptureC_'+chr+'.npy')
    features = np.load(DATADIR_features+'features_'+chr+'.npy')
    cca_train = cca.train([demean(CaptureC), demean(features)])
    np.save(DATADIR_weights+'ws_NoKernel_'+chr, cca_train.__dict__['ws'][1])

Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 components
Training CCA, regularization = 0.0010, 50 comp

## Save predictions, 2d

In [25]:
for chr in all_chrs:
    print chr
    without_current_chr = copy.copy(all_chrs)
    without_current_chr.remove(chr)
    ws_all = []
    for other_chr in without_current_chr:
        ws_all.append(np.load(DATADIR_weights+'ws_NoKernel_'+other_chr+'.npy'))
    ws = sum(ws_all)/len(without_current_chr) 
    features = np.load(DATADIR_features+'features_'+chr+'.npy')
    preds, projs = cca.predict_from_features(features, ws)
    np.save(DATADIR_preds+'pred_2d_NoKernel_'+chr, set_diag_to_zero(preds))

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX
